## 🎯 YOLO Pose Estimation Demo

이 노트북에서는 학습된 YOLO Pose 모델을 사용하여
1. 📸 **이미지**에서 포즈 예측
2. 📹 **웹캠**으로 실시간 포즈 예측

---
#### 1️⃣  필요한 라이브러리 임포트

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')

from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np
from IPython.display import display, clear_output


print("✅ 라이브러리 로드 완료!")

---
#### 2️⃣ YOLO 모델 로드

- 학습한 `모델의 경로`를 입력해 보세요

In [ ]:
# 모델 경로 설정
# ex) MODEL_PATH = 'runs/pose/train/weights/best.pt'
MODEL_PATH = ''

# 모델 로드
try:
    model = YOLO(MODEL_PATH)
    print(f"✅ 모델 로드 성공: {MODEL_PATH}")
    print(f"📊 모델 정보: {model.info()}")
except Exception as e:
    print(f"❌ 모델 로드 실패: {e}")
    print("💡 모델 경로를 확인해주세요!")

---
#### 📸 Part 1: 이미지에서 포즈 예측

단일 이미지 파일에서 포즈를 감지합니다.

In [ ]:
# 이미지 경로 설정 (본인의 이미지 경로로 변경하세요)
# image_path = '../data_10%/demo/demo_1.jpg'
image_path = ''  # ← 여기에 테스트할 이미지 경로를 입력하세요

# 이미지 예측
if os.path.exists(image_path):
    results = model.predict(source=image_path, conf=0.5, save=False, verbose=False)
    # 결과 시각화
    for result in results:
        
        img_with_pose = result.plot()  # 포즈가 그려진 이미지

        plt.figure(figsize=(4,4))
        plt.imshow(cv2.cvtColor(img_with_pose, cv2.COLOR_BGR2RGB))
        plt.axis('off')
        plt.title('Pose Estimation Result', fontsize=16)
        plt.tight_layout()
        plt.show()
        
        # 감지된 keypoints 정보 출력
        if result.keypoints is None:
            print("\n❌ 포즈가 감지되지 않았습니다.")      
else:
    print(f"❌ 이미지 파일을 찾을 수 없습니다: {image_path}")
    print("💡 image_path 변수를 올바른 이미지 경로로 설정해주세요!")

---
#### 📹 Part 2: 웹캠으로 실시간 포즈 예측

웹캠을 사용하여 실시간으로 포즈를 감지합니다.

⚠️ **주의사항:**
- 웹캠이 연결되어 있어야 합니다.
- Jupyter 환경에 따라 실행이 안 될 수 있습니다.
- 종료하려면 'q' 키를 누르세요.

In [ ]:
def run_webcam_pose_detection(conf_threshold=0.5):
    """
    웹캠으로 실시간 포즈 감지
    
    Args:
        conf_threshold: confidence threshold (기본값: 0.5)
    
    종료: 'q' 키를 누르세요
    """
    cap = cv2.VideoCapture(0)  # 0은 기본 웹캠
    
    if not cap.isOpened():
        print("❌ 웹캠을 열 수 없습니다.")
        print("💡 웹캠이 연결되어 있는지 확인하세요!")
        return
    
    print("\n🎥 웹캠 실행 중...")
    print("📍 종료하려면 'q' 키를 누르세요!")
    print("📍 스크린샷을 찍으려면 's' 키를 누르세요!\n")
    
    frame_count = 0
    screenshot_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("❌ 프레임을 읽을 수 없습니다.")
                break
            
            # YOLO 예측
            results = model.predict(source=frame, conf=conf_threshold, save=False, verbose=False)
            
            # 결과 이미지 가져오기
            annotated_frame = results[0].plot()
            
            # FPS 계산 및 표시
            frame_count += 1
            cv2.putText(annotated_frame, f'Frame: {frame_count}', (10, 30),
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # 감지된 사람 수 표시
            if results[0].keypoints is not None:
                num_people = len(results[0].keypoints)
                cv2.putText(annotated_frame, f'People: {num_people}', (10, 70),
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            
            # 화면에 표시
            cv2.imshow('YOLO Pose Detection - Press Q to quit, S for screenshot', annotated_frame)
            
            # 키 입력 처리
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                print("\n👋 웹캠 종료")
                break
            elif key == ord('s'):
                # 스크린샷 저장
                screenshot_filename = f'screenshot_{screenshot_count:03d}.jpg'
                cv2.imwrite(screenshot_filename, annotated_frame)
                print(f"📸 스크린샷 저장: {screenshot_filename}")
                screenshot_count += 1
    
    except KeyboardInterrupt:
        print("\n⚠️ 사용자가 중단했습니다.")
    
    finally:
        cap.release()
        cv2.destroyAllWindows()
        print(f"\n✅ 총 {frame_count} 프레임 처리 완료")
        print(f"📸 총 {screenshot_count}개 스크린샷 저장")

# 웹캠 실행
# run_webcam_pose_detection(conf_threshold=0.5)